<a href="https://colab.research.google.com/github/KalanaBimsara/Bank-Client-Prediction/blob/main/CW_ML_CM2604.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout




In [6]:
import pandas as pd

# Load the dataset
file_path = '/mnt/data/bank-additional-full.csv'
data = pd.read_csv(file_path, sep=';')  # The dataset uses a semicolon as the delimiter

# Display the first few rows and basic info
data_info = data.info()
data_head = data.head()

data_info, data_head


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
# Check for missing values
print(data.isnull().sum())

# Encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    if column != 'y':  # Skip target variable for now
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le

# Encode target variable (y)
data['y'] = data['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Scale numerical features
scaler = StandardScaler()
numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Split the data into training and test sets
X = data.drop('y', axis=1)
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


In [ ]:
# Ensure the target variable is encoded correctly
data['y'] = data['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Split the data again
X = data.drop('y', axis=1)
y = data['y']

# Ensure target is binary before splitting
print(f"Unique values in target variable: {y.unique()}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


Unique values in target variable: [0]


In [ ]:
# Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
# Evaluate the model
rf_preds = rf_model.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_preds))


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8238

    accuracy                           1.00      8238
   macro avg       1.00      1.00      1.00      8238
weighted avg       1.00      1.00      1.00      8238



In [ ]:
# Build a Neural Network
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
nn_preds = (nn_model.predict(X_test) > 0.5).astype(int)
print("Neural Network Classification Report:")
print(classification_report(y_test, nn_preds))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
824/824 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9631 - loss: 0.0918 - val_accuracy: 1.0000 - val_loss: 5.0953e-05
Epoch 2/20
824/824 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 1.0000 - loss: 1.6451e-04 - val_accuracy: 1.0000 - val_loss: 7.9691e-06
Epoch 3/20
824/824 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 1.0000 - loss: 5.0944e-05 - val_accuracy: 1.0000 - val_loss: 2.2900e-06
Epoch 4/20
824/824 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 2.8705e-05 - val_accuracy: 1.0000 - val_loss: 7.8492e-07
Epoch 5/20
824/824 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 1.5872e-05 - val_accuracy: 1.0000 - val_loss: 3.0510e-07
Epoch 6/20
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 7.6924e-06 - val_accuracy: 1.0000 - val_loss: 1.2909e-07
Epoch 7/20
824/824 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 1.0000 - loss: 5.0763e-06 - val_accuracy: 1.0000 - val_loss: 5.4433e-08
Epoch 8/20
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step 

In [ ]:
# Compare Accuracy
rf_accuracy = accuracy_score(y_test, rf_preds)
nn_accuracy = accuracy_score(y_test, nn_preds)

print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Neural Network Accuracy: {nn_accuracy}")

# Plot ROC Curves
rf_probs = rf_model.predict_proba(X_test)[:, 1]
nn_probs = nn_model.predict(X_test).flatten()

rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_probs)
nn_fpr, nn_tpr, _ = roc_curve(y_test, nn_probs)

plt.figure(figsize=(10, 6))
plt.plot(rf_fpr, rf_tpr, label='Random Forest (AUC = {:.2f})'.format(roc_auc_score(y_test, rf_probs)))
plt.plot(nn_fpr, nn_tpr, label='Neural Network (AUC = {:.2f})'.format(roc_auc_score(y_test, nn_probs)))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


Random Forest Accuracy: 1.0
Neural Network Accuracy: 1.0


IndexError: index 1 is out of bounds for axis 1 with size 1